## Diffusive Flux Matrix Implementation

The diffusive matrix is related to the viscous and thermal properties of the fluid. The former are presented both in the conservation of momentum and energy, while the latter are necessary to define the conservation of energy only.

The shear stress tensor is defined as follow:

\begin{equation}
    \tau_{ij} (\mathbf{u}) = \mu \bigg(\frac{\partial u_i}{\partial x_j} + \frac{\partial u_j}{\partial x_i} \bigg) -\frac{2 \mu}{3} \bigg(\frac{\partial u_l}{\partial x_l} \bigg) \delta_{ij}
\end{equation}

with the dynamic viscosity $\mu$. The relation can be rewritten in terms of conservative variables as:

\begin{equation}
    \tau_{ij}(\mathbf{U}) = \frac{\mu}{\rho}\bigg(\frac{\partial m_i}{\partial x_j}+\frac{\partial m_j}{\partial x_i}\bigg)-\frac{2\mu}{3\rho}\bigg(\frac{\partial m_k}{\partial x_k}\bigg)\delta_{ij}-\frac{\mu}{\rho^2}\bigg(m_i \frac{\partial\rho}{\partial x_j}+m_j \frac{\partial \rho}{\partial x_i}\bigg)+\frac{2\mu}{3\rho^2}\bigg(m_k \frac{\partial \rho}{\partial x_k}\bigg)\delta_{ij} \quad i,j,k<{d}
\end{equation}

Substituting $\mathbf{H}(\mathbf{U})= \frac{\partial \mathbf{U}}{\partial x_j}$, which is a $\textit{(dim+2)*(dim)}$ matrix, and $\mathbf{U}$ in the equation:

\begin{equation}
    \tau_{ij}(\mathbf{U}) = \frac{\mu}{U_0}\bigg(H_{i+1,j}+H_{j+1,i}\bigg)-\frac{2\mu}{3U_0}\bigg(H_{k+1,k}\bigg)\delta_{ij}-\frac{\mu}{U_0^2}\bigg(U_{i+1} H_{0,j}+U_{j+1}H_{0,i}\bigg)+\frac{2\mu}{3U_0^2}\bigg(U_{k+1} H_{0,k}\bigg)\delta_{ij} \quad i,j,k<{d}
\end{equation}

The heat flux vector $q_i(\mathbf{U})$ is defined by the Fourier's law as:

\begin{equation}
    q_i(\theta) = -\lambda \frac{\partial \theta}{\partial x_i}
\end{equation}

where $\lambda$ is the thermal conductivity and $\theta$ is the temperature of the fluid.

Two additional equations are necessary in order to close the problem. Here the caloric equation and the perfect gas state equation are used in order to compute the pressure $\textit{p}$ and the speed of sound $\textit{c}$.

\begin{equation}
    e = c_v(\theta)\theta \quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad p = \rho R \theta
\end{equation}

Where $\textit{R} = c_p - c_v$ and $\gamma = \frac{c_p}{c_v}$ is the ratio between the specific heat at constant pressure $c_p$ and the specific heat at constant volume $c_v$. The internal energy can be expressed in terms of the conservative variables leading to the following equation for the heat flux vector.

\begin{equation}
    e = \frac{e_{tot}}{\rho}- \frac{|\textbf{m}|^2}{2}
\end{equation}


\begin{equation}
    q_i(\mathbf{U}) = \frac{\lambda \textit{e}_{tot}}{\rho^2 \textit{c}_v}\frac{\partial \rho}{\partial x_i}-\frac{\lambda m_jm_j}{\rho^3 \textit{c}_v}\frac{\partial \rho}{\partial x_i}+\frac{\lambda m_j}{\rho^2 \textit{c}_v}\frac{\partial m_j}{\partial x_i}-\frac{\lambda}{\rho \textit{c}_v}\frac{\partial \textit{e}_{tot}}{\partial x_i}
\end{equation}

Which substituting $\mathbf{H}$ and $\mathbf{U}$ becomes:
\begin{equation}
    q_i(\mathbf{U}) = \frac{\lambda U_{d+1}}{U_0^2 \textit{c}_v}H_{0,i}-\frac{\lambda U_{j+1}U_{j+1}}{U_0^3 \textit{c}_v}H_{0,i}+\frac{\lambda U_{j+1}}{U_0^2 \textit{c}_v}H_{j+1,i}-\frac{\lambda}{U_0 \textit{c}_v}H_{d+1}\quad i,j<d
\end{equation} 

NB: $\rho$ is the first term of the vector of unknwons $\mathbf{U}$ and $\textit{e}_{tot}$ is the last one (Ug\[dim+1\]).
Remember to consider them as Unknowns and not fluid parameters.

Defining $\mathbf{G}$ as:

\begin{equation}
    \label{Gmat}
    \mathbf{G}_j(\mathbf{U}) = \big(0,-\tau_{ji}, -\frac{m_i}{\rho}\tau_{ij}+q_j\big)^T \quad \quad i,j<{d}
\end{equation}

\begin{equation}
    \mathbf{G} =
  \begin{bmatrix}
    0 & 0 & 0 \\
    -\tau_{11} & -\tau_{21} & -\tau_{31}\\
    -\tau_{12} & -\tau_{22} & -\tau_{32}\\
    -\tau_{13} & -\tau_{23} & -\tau_{33}\\
    -\frac{m_1}{\rho}\tau_{11}-\frac{m_2}{\rho}\tau_{21}-\frac{m_3}{\rho}\tau_{31}+q_1 & 
   -\frac{m_1}{\rho}\tau_{12}-\frac{m_2}{\rho}\tau_{22}-\frac{m_3}{\rho}\tau_{32}+q_2 &
   -\frac{m_1}{\rho}\tau_{13}-\frac{m_2}{\rho}\tau_{23}-\frac{m_3}{\rho}\tau_{33}+q_3
  \end{bmatrix}
\end{equation}

The fourth order tensor $\mathbf{K}_{kj}(\mathbf{U})$ $\textit{(d+2)*(d+2)*d*d}$ can be manually derived in order to satisfy this expression:

\begin{equation}
    \label{K}
    \frac{\partial\mathbf{G}_j(\mathbf{U})}{\partial x_j} =- \frac{\partial}{\partial x_k}\bigg(\mathbf{K}_{kj}(\mathbf{U})\frac{\partial \mathbf{U}}{\partial x_j}\bigg)
\end{equation}

In this project the $\mathbf{K}$ tensor was not computed. 

In [2]:
from KratosMultiphysics import *

from sympy import *
from sympy_fe_utilities import *
import pprint

## Computation of the Diffusive Matrix
def computeK(dofs,params,Hg,Gg):
    print("\nCompute Diffusive Matrix\n")
    dim = params["dim"]				                    # spatial dimensions
    
    ## Unknown fields definition
    H = Hg.copy()                               		# Gradient of U
    G = Gg.copy()                               		# Diffusive Flux matrix 
    tau_stress = DefineMatrix('tau_stress',dim,dim)		# Shear stress tensor for Newtonian fluid
    q = DefineVector('q',dim)			# Heat flux vector
    
    ## Other simbols definition
    c_v = params["c_v"]				    # Specific Heat at Constant volume
    gamma = params["gamma"]				# Gamma (Cp/Cv) 
    mu  = params["mu"]         			# Dynamic viscosity 
    l = params["lambda"]			    # Thermal Conductivity of the fluid
        
    ## Data interpolation to the Gauss points
    Ug = dofs             

    ## Pgauss - Pressure definition
    pg = Ug[dim+1]
    for i in range(0,dim):
        pg += (-Ug[i+1]*Ug[i+1]/(2*Ug[0]))
    pg *= (gamma-1)

    ## Tau - Shear stress tensor definition
    for i in range(0,dim):
        for j in range(i,dim):
            if i!=j:
               tau_stress[i,j] = (mu/Ug[0])*(H[i+1,j]+H[j+1,i])-(mu/Ug[0]**2)*(Ug[i+1]*H[0,j]+Ug[j+1]*H[0,i])
            if i==j:
               tau_stress[i,j]= (2*mu/Ug[0])*H[i+1,i]-(2*mu/Ug[0]**2)*Ug[i+1]*H[0,i]
               for k in range(0,dim):
                   tau_stress[i,j]+= -(2*mu/(3*Ug[0]))*H[k+1,k]+(2*mu/(3*Ug[0]**2))*Ug[k+1]*H[0,k]
    
    for i in range(1,dim):
        for j in range(0,dim-1):
            if j!=i:
               tau_stress[i,j] = tau_stress[j,i]
               
    ## q - Heat flux vector definition
    for i in range(0,dim):
        q[i] = l*Ug[dim+1]/(Ug[0]**2*c_v)*H[0,i]-(l*H[dim+1,i])/(Ug[0]*c_v)
        for j in range(0,dim):
            q[i] += -l*Ug[j+1]**2/(c_v*Ug[0]**3)*H[0,i]+l/(Ug[0]**2*c_v)*Ug[j+1]*H[j+1,i] 
    #NB!!!There is an error in the definition of q[i] in the research proposal. 
    #The second term of the equation has an opposite sign!!!NB#
    ''' 
    G [(dim+2)*(dim)]
    
    0                                   0 
    -tau00                              -tau01
    -tau01                              -tau11
    -mu/rho*tau00-mv/rho*tau01+q0       -mu/rho*tau01-mv/rho*tau11+q1
    '''
    ## G - Diffusive Matrix definition 
    for j in range(0,dim):
        G[0,j]= 0 			                #Mass equation related
       
    for i in range(1,dim+1):
        for j in range(0,dim):
            G[i,j]=-tau_stress[i-1,j]		#Moment equation related
    
    for j in range(0,dim):                  #Energy equation related
        G[dim+1,j] = q[j]
        for k in range(0,dim):
            G[dim+1,j] += -Ug[k+1]*tau_stress[k,j]/Ug[0]

    return G
    
## Printing the Diffusive Matrix G
def printK(G,params):
    dim = params["dim"]
    print("The diffusive matrix is:\n")
    for ll in range(dim+2):
        for mm in range(dim+2):
            print("G[",ll,",",mm,"]=",G[ll,mm],"\n")
       
    return 0
